In [6]:
!pip install spacy
!pip install experta


The system cannot find the file C:\Users\Bizzl\AppData\Local\Microsoft\WindowsApps\pip.exe.
The system cannot find the file C:\Users\Bizzl\AppData\Local\Microsoft\WindowsApps\pip.exe.


In [7]:
import random
import json
import spacy
import difflib
from difflib import get_close_matches, SequenceMatcher
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

from random import choice
from experta import *

ModuleNotFoundError: No module named 'spacy'

In [ ]:
# You may need to restart the kernel after running these two commands for it to work.
import spacy.cli
spacy.cli.download("en_core_web_sm")

ModuleNotFoundError: No module named 'spacy'

In [ ]:
# If you are using Colab, you should upload data folder and use its path here
intentions_path = "intentions.json"


In [ ]:
import json 

In [ ]:
# Opening JSON file and return JSON object as a dictionary
with open(intentions_path) as f:
    intentions = json.load(f)

print(json.dumps(intentions, indent=4))

{
    "greeting": {
        "patterns": [
            "hi",
            "hello",
            "hey",
            "holla"
        ],
        "responses": [
            "Hello! How can I assist you with your travel plans today?",
            "Hi there! Looking to book a ticket?"
        ]
    },
    "goodbye": {
        "patterns": [
            "bye",
            "goodbye",
            "see you",
            "later"
        ],
        "responses": [
            "Have a great trip! Bye.",
            "Thank you for using our service. Goodbye!"
        ]
    },
    "thanks": {
        "patterns": [
            "thanks",
            "thank you",
            "thx"
        ],
        "responses": [
            "You're very welcome! Glad to assist you.",
            "Happy to help with your travel needs!"
        ]
    },
    "cancellation_policy": {
        "patterns": [
            "How can I cancel my ticket",
            "Cancellation policy",
            "Can I cancel my ticket"
        ]

In [ ]:
import random 

In [ ]:
final_chatbot = False

def check_intention_by_keyword(sentence):
    for word in sentence.split():
        for type_of_intention in intentions:
            if word.lower() in intentions[type_of_intention]["patterns"]:
                print("BOT: " + random.choice(intentions[type_of_intention]["responses"]))
                
                # Do not change these lines
                if type_of_intention == 'greeting' and final_chatbot:
                    print("BOT: We can talk about the time, date, and train tickets.")
                return type_of_intention
    return None

sample_user_input = "How can I cancel my ticket"
print(sample_user_input)
print(f'Detected intention: {check_intention_by_keyword(sample_user_input)}')
print('*' * 50)
sample_user_input = "Thank you very much!"
print(sample_user_input)
print(f'Detected intention: {check_intention_by_keyword(sample_user_input)}')

How can I cancel my ticket
Detected intention: None
**************************************************
Thank you very much!
Detected intention: None


In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")


In [ ]:
def lemmatize_and_clean(text):
    doc = nlp(text.lower())
    out = ""
    for token in doc:
        if not token.is_punct and not token.is_stop:
            out = out + token.lemma_ + " "
    return out.strip()

sample_user_input = "Tell me the time!"
lemmatize_and_clean(sample_user_input)

'tell time'

In [ ]:
import difflib

In [ ]:
tickets = [
    {"start_loc": "Norwich", "end_loc": "Manchester", "date": "2024-06-05", "time": "09:45", "price": 150},
    {"start_loc": "Norwich", "end_loc": "Manchester", "date": "2024-06-05", "time": "11:00", "price": 140},  # Cheapest for Manchester
    {"start_loc": "Norwich", "end_loc": "Birmingham", "date": "2024-06-05", "time": "09:00", "price": 130},
    {"start_loc": "Norwich", "end_loc": "Birmingham", "date": "2024-06-05", "time": "10:15", "price": 125},  # Cheapest for Birmingham
    {"start_loc": "Norwich", "end_loc": "Edinburgh", "date": "2024-06-05", "time": "08:30", "price": 220},
    {"start_loc": "Norwich", "end_loc": "Edinburgh", "date": "2024-06-05", "time": "12:00", "price": 210}   # Cheapest for Edinburgh
]

def find_cheapest_ticket(start_loc, end_loc, date, time=None):
    # Filter tickets by start location, end location, and date
    filtered_tickets = [ticket for ticket in tickets if ticket['start_loc'].lower() == start_loc.lower() and
                        ticket['end_loc'].lower() == end_loc.lower() and
                        ticket['date'] == date]
    
    # If a specific time is provided, further filter the tickets
    if time:
        filtered_tickets = [ticket for ticket in filtered_tickets if ticket['time'] == time]

    # Find the cheapest ticket from the filtered list
    if filtered_tickets:
        cheapest_ticket = min(filtered_tickets, key=lambda x: x['price'])
        return f"Cheapest ticket from {start_loc} to {end_loc} on {date} at {cheapest_ticket['time']} is {cheapest_ticket['price']} Pounds."
    else:
        return "No tickets available for the specified criteria."


Cheapest ticket from Norwich to New York on 2024-06-05 at 09:45 is 120 Pounds.
Cheapest ticket from Norwich to London on 2024-06-05 at 12:00 is 180 Pounds.


In [ ]:
from spacy.matcher import Matcher

nlp = spacy.load('en_core_web_sm')
matcher = Matcher(nlp.vocab)

# Define pattern
pattern = [{"LOWER": "from"}, {"ENT_TYPE": "GPE", "OP": "+"}, {"LOWER": "to"}, {"ENT_TYPE": "GPE", "OP": "+"}]
matcher.add("TRAVEL_ROUTE", [pattern])

def extract_travel_route(text):
    doc = nlp(text)
    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]  # The matched span
        return span.text
    return None

# Example usage
route = extract_travel_route("I need a ticket from New York to London next Monday")
print(route)

from New York to London


In [ ]:
def parse_route(route):
    if not route:
        return None, None

    # Split the route into start and end parts
    parts = route.split(' to ')
    if len(parts) < 2:
        return None, None

    # Extracting start location more cleanly
    start_loc = parts[0].strip()
    if start_loc.lower().startswith("from "):
        start_loc = start_loc[5:].strip()  # Remove 'from ' prefix

    # Extracting end location
    end_loc = parts[1].strip()

    return start_loc, end_loc

In [ ]:
def handle_user_query(text):
    route = extract_travel_route(text)
    start_loc, end_loc = parse_route(route)
    if start_loc and end_loc:
        # Assuming the date is hardcoded for the example
        return find_cheapest_ticket(start_loc, end_loc, "2024-06-05")
    else:
        return "Could not extract both start and end locations properly. Please check your input."
    return False

# Test the handling function
response = handle_user_query("I need to book a train from Norwich to London.")
print(response)  # Should return the cheapest ticket information or an error message

Cheapest ticket from Norwich to London on 2024-06-05 at 12:00 is 180 Pounds.


In [ ]:
pip install experta

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from experta import Fact, KnowledgeEngine, Rule, AS, L


In [ ]:
final_chatbot = False

class Book(Fact):
    """Info about the booking ticket."""
    pass

class TrainBot(KnowledgeEngine):
  @Rule(Book(ticket='one way'))
  def one_way(self):
    print("BOT: You have selected a one way ticket. Have a good trip.")
    if final_chatbot:
      print("BOT: If you don't have any other questions you can type bye.")

  @Rule(Book(ticket='round'))
  def round_way(self):
    print("BOT: You have selected a round ticket. Have a good trip.")
    if final_chatbot:
      print("BOT: If you don't have any other questions you can type bye.")

  @Rule(AS.ticket << Book(ticket=L('open ticket') | L('open return')))
  def open_ticket(self, ticket):
    print("BOT: You have selected a " + ticket["ticket"] +".  Have a good trip.")
    if final_chatbot:
      print("BOT: If you don't have any other questions you can type bye.")

In [ ]:
from random import choice


In [ ]:
engine = TrainBot()
engine.reset()
engine.declare(Book(ticket=choice(['one way', 'round', 'open ticket', 'open return'])))
engine.run()

BOT: You have selected a open return.  Have a good trip.


In [ ]:
def check_ticket(user_input):
  user_input = user_input.lower()
  ticket_list = ['one way', 'round', 'open ticket', 'open return']    
  
  for ticket in ticket_list:
    if ticket in user_input:
      return ticket_list[ticket_list.index(ticket)]
  
  return None
  

def expert_response(user_input):
    engine = TrainBot()
    engine.reset()
    ticket = check_ticket(user_input)
    if ticket != None:
        engine.declare(Book(ticket=ticket))
        engine.run()
        return True
    
    return False

sample_user_input = "I want a round ticket."
print(sample_user_input)
expert_response(sample_user_input)

I want a round ticket.
BOT: You have selected a round ticket. Have a good trip.


True

In [ ]:
final_chatbot = True
flag = True
print("BOT: Hi there! How can I help you?.\n (If you want to exit, just type bye!)")

while(flag):
    user_input = input("You: ")
    #print(f"Debug: Received input {user_input}")  # Debug input

    intention = check_intention_by_keyword(user_input)
    #print(f"Debug: Intention found {intention}")  # Debug intention

    if intention == 'goodbye':
        print("BOT: Goodbye! Have a great day!")
        flag = False
    elif intention is None:
        handled_by_expert = expert_response(user_input)
       #print({handled_by_expert})  # Debug expert handling

        if not handled_by_expert:
            handled_by_query = handle_user_query(user_input)
            print({handled_by_query})  # Debug query handling

            if not handled_by_query:
                print("BOT: Sorry I don't understand that. Please rephrase your statement.")


BOT: Hi there! How can I help you?.
 (If you want to exit, just type bye!)
BOT: Hello! How can I assist you with your travel plans today?
BOT: We can talk about the time, date, and train tickets.
{'Cheapest ticket from Norwich to London on 2024-06-05 at 12:00 is 180 Pounds.'}
BOT: Happy to help with your travel needs!
BOT: Have a great trip! Bye.
BOT: Goodbye! Have a great day!
